In [2]:
!pip install soynlp
!pip show soynlp

    100% |████████████████████████████████| 1.5MB 3.3MB/s ta 0:00:01
Name: soynlp
Version: 0.0.46
Summary: Unsupervised Korean Natural Language Processing Toolkits
Home-page: https://github.com/lovit/soynlp
Author: Lovit
Author-email: soy.lovit@gmail.com
License: UNKNOWN
Location: /anaconda3/lib/python3.6/site-packages
Requires: numpy, psutil
Required-by: 


In [3]:
import pandas as pd 
import numpy as np
import re
from soynlp.tokenizer import RegexTokenizer
import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import corpora, models
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus

In [0]:
# 참고자료 
# http://nbviewer.jupyter.org/github/KMJJ1/hiphop/blob/master/RNN%20-%20LSTM%EC%9D%84%20%EC%9D%B4%EC%9A%A9%ED%95%9C%20Hiphop%20%EA%B0%80%EC%82%AC%20%EC%83%9D%EC%84%B1.ipynb
# https://github.com/dyelax/encore.ai

In [4]:
%pwd

'/Users/lab/Desktop/Word2Vec_Idol'

In [5]:
data= pd.read_csv("song_data_yewon_ver05.csv")

In [6]:
song = pd.DataFrame(data)

In [7]:
song.head(3)

,title,artist,album,release_date,song_genre,is_title,like,creator,lyrics,lyricist,composer,arranger
0,태지 보이스,서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,수록곡,269,서태지 작사 서태지 작곡 서태지 편곡,갈수 없는 곳이란 우리에겐 없어<br>보이는 길밖에도 세상은 있어<br>언제나 식지...,서태지,서태지,서태지
1,시대유감(時代遺憾),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,타이틀 곡,"3,265",서태지 작사 서태지 작곡 서태지 편곡,왜 기다려 왔잖아<br>모든 삶을 포기하는 소리를<br>이 세상이 모두 미쳐버릴<b...,서태지,서태지,서태지
2,수시아(誰是我),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,수록곡,488,서태지 작사 서태지 작곡 서태지 편곡,이 세상 그 누구도 나를 닮을 순 없네<br>날 세상에 알릴 거야<br>나 역시 그...,서태지,서태지,서태지


In [8]:
song.columns

Index(['title', 'artist', 'album', 'release_date', 'song_genre', 'is_title',
       'like', 'creator', 'lyrics', 'lyricist', 'composer', 'arranger'],
      dtype='object')

In [9]:
song['artist'].value_counts().head()

슈퍼주니어          261
쿨              235
신화             225
동방신기           192
플라이 투 더 스카이    180
Name: artist, dtype: int64

In [10]:
# 유니크한 가수의 수 
len(song['artist'].unique())

322

In [11]:
# 곡의 수 
song.shape

(9861, 12)

In [12]:
song['lyrics'].isnull().sum()

0

In [69]:
song['lyrics'].head()

0    갈수 없는 곳이란 우리에겐 없어<br>보이는 길밖에도 세상은 있어<br>언제나 식지...
1    왜 기다려 왔잖아<br>모든 삶을 포기하는 소리를<br>이 세상이 모두 미쳐버릴<b...
2    이 세상 그 누구도 나를 닮을 순 없네<br>날 세상에 알릴 거야<br>나 역시 그...
3    항상 난 지나치기만 했네<br>이젠 난 미련없이 끝내<br>됐어 찾아냈어 난<br>...
4    난 알아요이밤이 흐르고 흐르면 누군가가<br>나를 떠나버려야 한다는 그사실을<br>...
Name: lyrics, dtype: object

## 텍스트 데이터 전처리

In [14]:
def preprocessing(text):
    # 개행문자 제거
    text =  text.strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [15]:
sample_content = song['lyrics'][100]
type(sample_content)

str

In [16]:
sample_content = preprocessing(sample_content)
sample_content[:1000]

'Why Why Why Why Why Oh Baby br Why Why Why Why Why br 너의 잠꼬대 속에 br 애타게 찾는 이는 br 내가 아닌 B였어 br 그것도 모자라서 br 다른 사람도 아닌 br 왜 하필 내 친구 B를 찾는 거야 br 이건 말도 안돼 도대체 넌 뭐야 br 기막혀 웃음밖에 안나와 br  류 영화속의 한심한 주연이 br 바로 너였다니 믿어지지 않아 br Why Why Why Why br B를 찾는 거야 br Why Why Why Why br 하필 내 친구야 br B는 널 보지 않아 br 문제는 오직 너야 br 난 아직 너를 사랑해 br 넌 어떻게 할꺼야 br 태도를 분명히해 br 난 B가 아니야 A란 말이야 br 이건 말도 안돼 도대체 넌 뭐야 br 기막혀 웃음밖에 안나와 br  류 영화속의 한심한 주연이 br 바로 너였다니 믿어지지않아 br Why Why Why Why Why br 왜 너는 B를 찾는 거야 br Why Why Why Why Why br 하필 내 친구야 br  br My heart beats and as we br conversate adrenaline peaks br your smiles your style br it makes you unique br I wanted to take the br friendship that we had to br a different strat br because I didn t think that br that this kind of love br was all that br pharcyde she s passing me by br I can t help it from br wondering why br Why Why Why Why Why br 왜 너는 B를 찾는 거야 br Why Why Why Why Why br 하필 내 친구야 br Why Why Why Why Why br Why Why Why Why Why br Why Why Why Why br 왜 B를 찾는 거야'

In [17]:
# %time을 찍어주면 해당 코드를 실행할 때 걸리는 시간을 출력해 줍니다
%time sentences = song['lyrics'].apply(preprocessing)

CPU times: user 1.32 s, sys: 11.6 ms, total: 1.33 s
Wall time: 1.33 s


In [18]:
sentences.head()

0    갈수 없는 곳이란 우리에겐 없어 br 보이는 길밖에도 세상은 있어 br 언제나 식지...
1    왜 기다려 왔잖아 br 모든 삶을 포기하는 소리를 br 이 세상이 모두 미쳐버릴 b...
2    이 세상 그 누구도 나를 닮을 순 없네 br 날 세상에 알릴 거야 br 나 역시 그...
3    항상 난 지나치기만 했네 br 이젠 난 미련없이 끝내 br 됐어 찾아냈어 난 br ...
4    난 알아요이밤이 흐르고 흐르면 누군가가 br 나를 떠나버려야 한다는 그사실을 br ...
Name: lyrics, dtype: object

## Bigram 만들기

'말이 되는' 가사 생성에 도움받기 위해 자주 나오는 단어들을 묶어 어구로 만들어야 한다.

n-gram 이라는 기법으로 bigram(두 단어씩 묶기), trigram(세 단어씩 묶기) 등의 방법이 있다.

`bigram = gensim.models.Phrases(tokens, min_count=1, threshold=100)`이런식으로 만들 수 있다.

trigram을 하려면 bigram 으로 나온 것을 한번 더 Phrases에 넣고 돌려주는 방식으로 하면 된다.

// to create the bigrams
bigram_model = Phrases(unigram_sentences)

// apply the trained model to a sentence
 for unigram_sentence in unigram_sentences:                
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])

// get a trigram model out of the bigram
trigram_model = Phrases(bigram_sentences)

In [21]:
# 토크나이징 하기 전 sentences 로 txt파일 생성

USE_PREMADE_TEXT = False

text_filepath = 'all_lyrics_text.txt'
if not USE_PREMADE_TEXT:
    with open(text_filepath, 'w', encoding='utf-8') as f:
        for lyrics in sentences.values:
            if pd.isnull(lyrics): # null값 있다면 그 다음으로 넘어감
                continue
            f.write(lyrics + '\n')
else:
    assert path.exists(text_filepath)

In [22]:
# bigram model 저장 


USE_PREMADE_BIGRAM_MODEL = False

all_bigram_model_filepath = 'all_bigram_model'
all_sentences_normalized_filepath = 'all_lyrics_text.txt'

all_unigram_sentences = LineSentence(all_sentences_normalized_filepath)

if not USE_PREMADE_BIGRAM_MODEL:    
    
    all_bigram_model = Phrases(all_unigram_sentences) #phrase냐 아니냐를 판단해줌
    all_bigram_model.save(all_bigram_model_filepath)
    
else:
    all_bigram_model = Phrases.load(all_bigram_model_filepath)

In [23]:
%%time
USE_PREMADE_BIGRAM_SENTENCES = False

all_bigram_sentences_filepath = 'all_sentences_for_word2vec.txt'

if not USE_PREMADE_BIGRAM_SENTENCES:
    
    with open(all_bigram_sentences_filepath, 'w', encoding='utf-8') as f:
        for unigram_sentence in all_unigram_sentences:
            all_bigram_sentence = all_bigram_model[unigram_sentence]
            f.write(' '.join(all_bigram_sentence) + '\n')
else:
    assert path.exists(all_bigram_sentences_filepath)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 9.16 s, sys: 25 ms, total: 9.18 s
Wall time: 9.2 s


## Word2vec 모델 만들기

Skip-gram 방식으로 word2vec 하였다.

Skip-Gram 방식은 CBOW 방식과 반대로 특정한 단어로부터 문맥이 될 수 있는 단어를 예측한다. 

보통 입력 단어 주변의 kk개 단어를 문맥으로 보고 예측 모형을 만드는데 이 kk 값을 window size 라고 한다.


In [24]:
# word2vec 모델 학습에 로그를 찍을 수 있도록 합니다.

import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)

In [33]:
%%time
USE_PREMADE_WORD2VEC = False

all2vec_filepath = 'all_word2vec_model'

if not USE_PREMADE_WORD2VEC:
    
    lyrics_for_word2vec = LineSentence(all_bigram_sentences_filepath)

    all2vec = Word2Vec(lyrics_for_word2vec, size=50, window=5, min_count=1, sg=1)
    # sg=0 cbow 1=Skip-Gram Model
    # 100차원으로 가져옴 / 보통 20~100 정도
    # window = 5 앞 5개, 뒤 5개 단어를 보겠다는 뜻
    # window size 작을수록 문법적인 의미가 너무 중요해짐, 클수록 주제 지향적으로 문맥적인 정보를 많이 담게 됨    
    for _ in range(9):
        all2vec.train(lyrics_for_word2vec,total_examples=9861, epochs=1)
        
        
    all2vec.save(all2vec_filepath)
else:
    all2vec = Word2Vec.load(all2vec_filepath)
all2vec.init_sims()

2018-09-17 14:56:15,742 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-09-17 14:56:15,742 : INFO : collecting all words and their counts
2018-09-17 14:56:15,744 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-09-17 14:56:16,415 : INFO : collected 142657 word types from a corpus of 2464846 raw words and 9861 sentences
2018-09-17 14:56:16,416 : INFO : Loading a fresh vocabulary
2018-09-17 14:56:16,876 : INFO : min_count=1 retains 142657 unique words (100% of original 142657, drops 0)
2018-09-17 14:56:16,877 : INFO : min_count=1 leaves 2464846 word corpus (100% of original 2464846, drops 0)
2018-09-17 14:56:17,220 : INFO : deleting the raw counts dictionary of 142657 items
2018-09-17 14:56:17,222 : INFO : sample=0.001 downsamples 19 most-common words
2018-09-17 14:56:17,223 : INFO : downsampling leaves estimated 1885067 word corpus (76.5% of prior 2464846)
2018-09-17 14:56:17,617 : INFO : estimated required memory fo

2018-09-17 14:56:58,227 : INFO : EPOCH 1 - PROGRESS: at 57.14% examples, 325476 words/s, in_qsize 5, out_qsize 0
2018-09-17 14:56:59,252 : INFO : EPOCH 1 - PROGRESS: at 73.69% examples, 326676 words/s, in_qsize 5, out_qsize 0
2018-09-17 14:57:00,277 : INFO : EPOCH 1 - PROGRESS: at 89.66% examples, 327295 words/s, in_qsize 5, out_qsize 0
2018-09-17 14:57:00,843 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-17 14:57:00,847 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-17 14:57:00,897 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-17 14:57:00,898 : INFO : EPOCH - 1 : training on 2464846 raw words (1885198 effective words) took 5.8s, 327782 effective words/s
2018-09-17 14:57:00,899 : INFO : training on a 2464846 raw words (1885198 effective words) took 5.8s, 327657 effective words/s
2018-09-17 14:57:00,899 : WARNING : Effective 'alpha' higher than previous training cycles
2018-09-17 14:57:00,900 : INFO : 

2018-09-17 14:57:34,311 : INFO : EPOCH 1 - PROGRESS: at 89.66% examples, 333106 words/s, in_qsize 5, out_qsize 0
2018-09-17 14:57:34,895 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-17 14:57:34,899 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-17 14:57:34,935 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-17 14:57:34,936 : INFO : EPOCH - 1 : training on 2464846 raw words (1884994 effective words) took 5.7s, 332705 effective words/s
2018-09-17 14:57:34,937 : INFO : training on a 2464846 raw words (1884994 effective words) took 5.7s, 332585 effective words/s
2018-09-17 14:57:34,937 : WARNING : Effective 'alpha' higher than previous training cycles
2018-09-17 14:57:34,938 : INFO : training model with 3 workers on 142657 vocabulary and 50 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2018-09-17 14:57:35,944 : INFO : EPOCH 1 - PROGRESS: at 18.98% examples, 322934 words/s, in_qsize 5, out_qsiz

CPU times: user 4min 7s, sys: 1.21 s, total: 4min 8s
Wall time: 1min 26s


In [34]:
all2vec_filepath = 'all_word2vec_model'
all2vec = Word2Vec.load(all2vec_filepath)

2018-09-17 14:57:44,288 : INFO : loading Word2Vec object from all_word2vec_model
2018-09-17 14:57:45,010 : INFO : loading wv recursively from all_word2vec_model.wv.* with mmap=None
2018-09-17 14:57:45,011 : INFO : setting ignored attribute vectors_norm to None
2018-09-17 14:57:45,011 : INFO : loading vocabulary recursively from all_word2vec_model.vocabulary.* with mmap=None
2018-09-17 14:57:45,012 : INFO : loading trainables recursively from all_word2vec_model.trainables.* with mmap=None
2018-09-17 14:57:45,012 : INFO : setting ignored attribute cum_table to None
2018-09-17 14:57:45,013 : INFO : loaded all_word2vec_model


In [35]:
# 총 14만 2657개의 단어를 얻었다 

a = pd.DataFrame(all2vec.wv.index2word)
len(a)

142657

In [36]:
len(a)

142657

## 100회 이상 등장한 단어들만 가지고 t-SNE 시각화

t-SNE란 차원축소 기법의 하나로 시각화를 위해 주로 사용된다

t-SNE는 고차원에서 가까운 포인트를 저차원에서도 계속 가깝게 두고, 먼 포인트는 계속 멀게 유지시킨다

데이터간의 유사도를 통해 데이터의 패턴을 찾을 수 있다

단어 중 100회 초과 등장한 단어들을 대상으로, word2vec로 얻어진 각 단어의 numpy 행렬값을 매핑시킨 후 2차원 그래프로 시각화하였다.


In [37]:
%%time
words = []
for i in (range(len(a))):
    cnt = all2vec.wv.vocab[a[0][i]] # 횟수 카운트
    if cnt.count > 100:
        words.append(a[0][i]) # 횟수 100회 초과하는 단어만 리스트화
    i += 1

CPU times: user 1.58 s, sys: 6.45 ms, total: 1.59 s
Wall time: 1.59 s


In [38]:
len (words)  # 100회 이상 등장한 단어는 2232개 

2232

In [39]:
# 100회 이상 등장한 단어들에 대해 vector값 가져오기

X = []
for i in words:
    X.append(all2vec.wv[i])

In [40]:
X2 = pd.DataFrame(X, index = words)

In [58]:
# 단어와 각 vector 값으로 데이터 프레임 생성
X2.head(10)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
br,-0.023778,0.213856,-0.182159,-0.081728,0.185672,0.165335,0.298478,-0.143742,0.466566,0.023884,...,0.171788,0.618985,-0.176627,0.203825,-0.215538,0.026004,0.016014,0.035542,-0.158892,0.184097
난,-0.121098,0.269243,-0.542368,-0.281212,0.282475,0.135464,0.411120,-0.494864,0.542521,-0.260719,...,0.127125,0.681417,-0.281216,-0.312138,-0.140526,-0.569822,-0.276762,0.286425,-0.183662,0.228344
내,-0.086487,0.077758,-0.363098,0.167812,0.417696,-0.031061,0.555183,-0.372585,0.305636,0.057293,...,-0.240544,0.697819,-0.821037,-0.269625,0.013455,-0.248477,0.418442,0.168628,-0.044590,0.020024
you,0.567178,0.349392,-0.668625,-0.532527,-0.367302,1.026607,0.602054,-0.326327,1.013028,-0.048605,...,-0.195361,1.107766,-0.270485,-0.278807,-0.068598,-0.629179,-0.041194,0.335897,-0.539835,0.015942
날,0.684609,0.233643,-0.675154,0.274687,0.448693,0.200026,0.560858,-0.533608,0.262289,0.007931,...,0.277761,0.776114,0.272793,0.017217,0.099921,0.637786,0.083665,-0.364012,-0.061177,0.034584
널,0.581870,-0.069966,-0.967504,0.575275,0.766563,0.075801,0.535210,-0.165211,0.776117,0.109946,...,0.228455,1.015002,0.406034,0.048769,-0.195290,-0.020796,-0.359621,0.449513,-0.097467,0.505283
내가,0.113316,0.045467,-0.490698,-0.499304,0.631978,-0.367080,-0.037340,0.112991,0.888338,-0.080551,...,0.129542,0.376456,0.071924,-0.629705,-0.511179,0.189828,0.179901,0.153443,-0.590409,0.001950
나,-0.080870,0.741188,-0.861362,-0.085301,0.733338,-0.125315,0.327789,-0.010028,0.542308,-0.027048,...,-0.109925,0.681255,0.413585,-0.881007,0.103152,-0.531105,-0.282482,0.005598,-0.601360,0.244590
넌,0.230829,-0.044347,0.037989,-0.153000,-0.166131,0.078545,0.456517,-0.332201,0.500992,0.461233,...,0.450883,0.920707,-0.578216,-0.294359,-0.081267,-0.320550,0.165420,0.490026,-0.243374,0.868386
그,0.682885,-0.056259,0.327360,0.068493,0.233073,0.009742,0.566693,-0.029091,0.900875,-0.112311,...,0.388665,0.465432,0.394259,-0.182538,0.029794,-0.260153,0.199687,-0.006659,0.573718,-0.114998


In [42]:
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

# t-SNE 모델 모델링 

USE_PREMADE_TSNE = False

tsne_filepath = 'tsne.pkl'

if not USE_PREMADE_TSNE:
    
    tsne = TSNE(random_state=0)
    tsne_points = tsne.fit_transform(X2)
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne_points, f)
else:
    with open(tsne_filepath, 'rb') as f:
        tsne_points = pickle.load(f)

tsne_df = pd.DataFrame(tsne_points, index=X2.index, columns=['x_coord', 'y_coord'])
tsne_df['word'] = tsne_df.index

In [43]:
# t-SNE 모델 시각화

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
output_notebook()

# prepare the data in a form suitable for bokeh.
plot_data = ColumnDataSource(tsne_df)

# create the plot and configure it
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   active_scroll='wheel_zoom'
                  )

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='orange')

# adjust visual elements of the plot
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# show 
show(tsne_plot);

Loading BokehJS ...

In [44]:
show(tsne_plot)

각 데이터 포인트에 마우스를 올리면 어떤 단어들이 함께 맵핑되어 있는지 확인 가능하다. 

- 진한 부분일 수록 비슷한 단어들이 많이 모여 있음을 알 수 있다.


- 한글, 영어가 끼리끼리 모여서 임베딩되었다. 



In [45]:
all2vec.most_similar(positive=['친구'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2018-09-17 15:01:47,602 : INFO : precomputing L2-norms of word weight vectors


[('범생이', 0.6625233292579651),
 ('꺽다리', 0.6543509364128113),
 ('홀쭉한', 0.6434130072593689),
 ('꼬맹이', 0.6384218335151672),
 ('술고래', 0.6373487114906311),
 ('새겼니', 0.635798454284668),
 ('애인이', 0.6265844106674194),
 ('예전부터', 0.6073660850524902),
 ('우정에서', 0.6041829586029053),
 ('탐한', 0.6016032099723816)]

In [46]:
all2vec.most_similar(positive=['여자는'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('남자들은', 0.7101030349731445),
 ('밖에_모르는', 0.6959201097488403),
 ('여자지', 0.6932433843612671),
 ('상황은', 0.6886695623397827),
 ('여잔', 0.6764312982559204),
 ('여자답게', 0.6690301895141602),
 ('걔는', 0.6682300567626953),
 ('여잔데', 0.6679635643959045),
 ('못_잊는', 0.6637659668922424),
 ('여자', 0.661081075668335)]

In [47]:
all2vec.most_similar(positive=['남자는'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('짐승', 0.7609363794326782),
 ('남잔', 0.7608175873756409),
 ('다르다고', 0.7495805025100708),
 ('여자라고', 0.7375447154045105),
 ('손해', 0.7368902564048767),
 ('애가', 0.7353352308273315),
 ('꺼내지도_마', 0.733702540397644),
 ('뻔하디_뻔한', 0.7284417152404785),
 ('걔', 0.7225561738014221),
 ('사람들은_말해', 0.7224016189575195)]

In [48]:
all2vec.most_similar(positive=['여자란'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('없다고_믿었던', 0.8485206365585327),
 ('있대도', 0.8480961918830872),
 ('헤픈', 0.8460044860839844),
 ('알릴', 0.8403252363204956),
 ('다고', 0.8365401029586792),
 ('살겠니', 0.8287050724029541),
 ('이유라고', 0.8267949819564819),
 ('다루는', 0.8246527910232544),
 ('믿지_못해', 0.8200684189796448),
 ('고민해봐야', 0.8197356462478638)]

In [53]:
all2vec.most_similar(positive=['여잔'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('잘해줄', 0.7440240979194641),
 ('착하고', 0.7320570945739746),
 ('남자', 0.7310494780540466),
 ('세상에서_나보다', 0.7271991968154907),
 ('이여자는', 0.7265884876251221),
 ('남자들이', 0.7168596386909485),
 ('빼곤', 0.7076917290687561),
 ('여자', 0.705070972442627),
 ('그게_되나', 0.7045426964759827),
 ('남자들과는', 0.702836811542511)]

In [50]:
all2vec.most_similar(positive=['남잔'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('남자는', 0.7608175873756409),
 ('남자', 0.7482824921607971),
 ('짐승', 0.7390589714050293),
 ('거_아냐', 0.7309236526489258),
 ('나빴어', 0.7254292368888855),
 ('같은_여잔', 0.7214054465293884),
 ('정말_나빴어', 0.7200878858566284),
 ('줄_알어', 0.7049037218093872),
 ('말하기', 0.6978828310966492),
 ('바람둥이', 0.6960875988006592)]

In [54]:
all2vec.most_similar(positive=['girl'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('Girl', 0.773356020450592),
 ('pretty_girl', 0.7523748874664307),
 ('gir_gir', 0.7517142295837402),
 ('gir', 0.750639796257019),
 ('baby', 0.7456661462783813),
 ('Oh_gorgeous', 0.7449946999549866),
 ('Korean', 0.7435527443885803),
 ('Maxican', 0.7411680221557617),
 ('really_good', 0.7367945313453674),
 ('bad_girl', 0.7362215518951416)]

In [55]:
all2vec.most_similar(positive=['boy'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('chu_chu', 0.7523370385169983),
 ('So_Sexy', 0.7494564652442932),
 ('my', 0.7459061145782471),
 ('handsome', 0.7457798719406128),
 ('little', 0.7406745553016663),
 ('반했나_봐', 0.7269704937934875),
 ('girl', 0.7260350584983826),
 ('Mamama', 0.7213174104690552),
 ('Hey_boy', 0.7203558683395386),
 ('너너너너너너너넌', 0.7192025780677795)]

In [56]:
all2vec.most_similar(positive=['예쁜'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('별로인', 0.7442902326583862),
 ('놀라겠지', 0.674453854560852),
 ('얼굴만', 0.6641553640365601),
 ('사랑스러운', 0.6625345945358276),
 ('얼굴은', 0.6489533185958862),
 ('도도해', 0.6328843235969543),
 ('해맑게', 0.6278266906738281),
 ('예쁘기만', 0.6271035075187683),
 ('케익위에', 0.6262555122375488),
 ('거짓말이라', 0.6236047148704529)]

In [59]:
all2vec.most_similar(positive=['몸매'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('쭉빠진', 0.7950356006622314),
 ('썬그라스', 0.7836313843727112),
 ('수영복에', 0.7812054753303528),
 ('바디라인', 0.7719101905822754),
 ('뿜어져', 0.7603346705436707),
 ('스타일', 0.7530008554458618),
 ('잘록한', 0.7482519745826721),
 ('뽐내', 0.7451620101928711),
 ('딱_봐도', 0.7444287538528442),
 ('비키니', 0.7432544827461243)]

In [63]:
all2vec.most_similar(positive=['이별'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('넌_쉽니', 0.7305760383605957),
 ('행복했잖아', 0.6712286472320557),
 ('인사는', 0.6669461131095886),
 ('만남', 0.6662607789039612),
 ('이름조차', 0.6646117568016052),
 ('놀이는', 0.6626908779144287),
 ('쉬울테니까', 0.6620108485221863),
 ('나와의', 0.6602398157119751),
 ('수집가이죠', 0.6573875546455383),
 ('다_흔들고선', 0.6547708511352539)]

## 최근노래로 갈 수록 가중치를 줘야 할까? 
최근 아이돌의 성향을 보고싶으면, 최근것만 보고 해봐야 할 것 같다. 

## LSTM 공부하고 학습 시켜보기 

코드참고 : https://github.com/KMJJ1/hiphop/blob/master/test_example.ipynb

Instructions to Train:
If you want to use our model to train your own artists, follow these steps:

Pick an artist – it should be someone with a lot of lyrics. (Over 100,000 words).
Collect all of the artist's lyrics from your favorite lyrics website. Save each song as a text file in data/artist_name/. We recommend leaving newlines in as a special token so that the network will learn line and stanza breaks.
Train by navigating to the code directory and running python runner.py -a <artist_name> -m <model_save_name>.
Our models were all trained for 30,000 steps.
Generate new songs by running 
python runner.py -a <artist_name> -l ../save/models/<model_save_name>/<ckpt_file> -t.
Optional: If you would like to specify "prime text" – the initial text that the model will generate from – pass in a string with the -p flag.
Share your trained models with us so we can feature them on our website! Create an issue with a link to a public Dropbox or Google Drive containing your model's .ckpt file.

RNN - LSTM으로 가사 생성¶

일반적인 뉴럴넷에 비해 RNN은 파라미터가 공유되며, backpropagation through time 이라는 특징이 있다
하지만 기존의 RNN은 긴 시퀀스를 잘 처리하지 못하며, 그라디언트가 소멸되는 현상이 발생함
LSTM이라는 RNN의 변형 모델이 등장
LSTM은 '게이트'를 통해 상태값을 계산 시 내부의 상태값을 얼마나 기억하고 사용할지 정한다
그라디언트 흐름을 좋게 만들고 베니싱 현상을 완화시켜 먼 과거에 대한 단어나 정보도 잘 기억할 수 있게 되었다
1) 1000곡의 가사로 학습시킴
Hiphop장르 1000곡의 가사를 RNN - LSTM 방식으로 학습
23만회 학습시킨 모델을 생성
초반 단어 3,4개를 입력하여 차후 나올 단어들을 예측

In [0]:
#!pip3 install tensorflow 
#!pip install --upgrade tensorflow

In [0]:
# -*- coding: utf-8 -*-
# Char-RNN 예제
# Author : solaris33
# Project URL : http://solarisailab.com/archives/2487
# GitHub Repository : https://github.com/solaris33/char-rnn-tensorflow/
# Reference : https://github.com/sherjilozair/char-rnn-tensorflow

import tensorflow as tf
import numpy as np

from utils import TextLoader

# 학습에 필요한 설정값들을 지정합니다.
data_dir = 'data/tinyshakespeare' # 셰익스피어 희곡 <리처드 3세> 데이터로 학습
#data_dir = 'data/linux' # <Linux 소스코드> 데이터로 학습
batch_size = 50 # Training : 50, Sampling : 1
seq_length = 50 # Training : 50, Sampling : 1
hidden_size = 128   # 히든 레이어의 노드 개수
learning_rate = 0.002
num_epochs = 2
num_hidden_layers = 2
grad_clip = 5   # Gradient Clipping에 사용할 임계값

# TextLoader를 이용해서 데이터를 불러옵니다.
data_loader = TextLoader(data_dir, batch_size, seq_length)
# 학습데이터에 포함된 모든 단어들을 나타내는 변수인 chars와 chars에 id를 부여해 dict 형태로 만든 vocab을 선언합니다.
chars = data_loader.chars 
vocab = data_loader.vocab
vocab_size = data_loader.vocab_size # 전체 단어개수

# 인풋데이터와 타겟데이터, 배치 사이즈를 입력받기 위한 플레이스홀더를 설정합니다.
input_data = tf.placeholder(tf.int32, shape=[None, None])  # input_data : [batch_size, seq_length])
target_data = tf.placeholder(tf.int32, shape=[None, None]) # target_data : [batch_size, seq_length])
state_batch_size = tf.placeholder(tf.int32, shape=[])      # Training : 50, Sampling : 1

# RNN의 마지막 히든레이어의 출력을 소프트맥스 출력값으로 변환해주기 위한 변수들을 선언합니다.
# hidden_size -> vocab_size
softmax_w = tf.Variable(tf.random_normal(shape=[hidden_size, vocab_size]), dtype=tf.float32)
softmax_b = tf.Variable(tf.random_normal(shape=[vocab_size]), dtype=tf.float32)

# num_hidden_layers만큼 LSTM cell(히든레이어)를 선언합니다.
cells = []
for _ in range(0, num_hidden_layers):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    cells.append(cell)

# cell을 종합해서 RNN을 정의합니다.
cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

# 인풋데이터를 변환하기 위한 Embedding Matrix를 선언합니다.
# vocab_size -> hidden_size
embedding = tf.Variable(tf.random_normal(shape=[vocab_size, hidden_size]), dtype=tf.float32)
inputs = tf.nn.embedding_lookup(embedding, input_data)

# 초기 state 값을 0으로 초기화합니다.
initial_state = cell.zero_state(state_batch_size, tf.float32)

# 학습을 위한 tf.nn.dynamic_rnn을 선언합니다.
# outputs : [batch_size, seq_length, hidden_size]
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=tf.float32)
# ouputs을 [batch_size * seq_length, hidden_size]] 형태로 바꿉니다.
output = tf.reshape(outputs, [-1, hidden_size])

# 최종 출력값을 설정합니다.
# logits : [batch_size * seq_length, vocab_size]
logits = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(logits)

# Cross Entropy 손실 함수를 정의합니다. 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_data))

# 옵티마이저를 선언하고 옵티마이저에 Gradient Clipping을 적용합니다.
# grad_clip(=5)보다 큰 Gradient를 5로 Clippin합니다.
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.apply_gradients(zip(grads, tvars))

# 세션을 열고 학습을 진행합니다.
with tf.Session() as sess:
    # 변수들에 초기값을 할당합니다.
    sess.run(tf.global_variables_initializer())
    
    for e in range(num_epochs):
        data_loader.reset_batch_pointer()
        # 초기 상태값을 지정합니다.
        state = sess.run(initial_state, feed_dict={state_batch_size : batch_size})

        for b in range(data_loader.num_batches):
            # x, y 데이터를 불러옵니다.
            x, y = data_loader.next_batch()
            # y에 one_hot 인코딩을 적용합니다. 
            y = tf.one_hot(y, vocab_size)            # y : [batch_size, seq_length, vocab_size]
            y = tf.reshape(y, [-1, vocab_size])       # y : [batch_size * seq_length, vocab_size]
            y = y.eval()

            # feed-dict에 사용할 값들과 LSTM 초기 cell state(feed_dict[c])값과 hidden layer 출력값(feed_dict[h])을 지정합니다.
            feed_dict = {input_data : x, target_data: y, state_batch_size : batch_size}
            for i, (c, h) in enumerate(initial_state):
                feed_dict[c] = state[i].c
                feed_dict[h] = state[i].h

            # 한스텝 학습을 진행합니다.
            _, loss_print, state = sess.run([train_step, loss, final_state], feed_dict=feed_dict)

            print("{}(학습한 배치개수)/{}(학습할 배치개수), 반복(epoch): {}, 손실함수(loss): {:.3f}".format(
                          e * data_loader.num_batches + b,
                          num_epochs * data_loader.num_batches,
                          (e+1), 
                          loss_print))

    print("트레이닝이 끝났습니다!")   
    

    # 샘플링 시작
    print("샘플링을 시작합니다!")
    num_sampling = 4000  # 생성할 글자(Character)의 개수를 지정합니다. 
    prime = u' '         # 시작 글자를 ' '(공백)으로 지정합니다.
    sampling_type = 1    # 샘플링 타입을 설정합니다.
    state = sess.run(cell.zero_state(1, tf.float32)) # RNN의 최초 state값을 0으로 초기화합니다.

    # Random Sampling을 위한 weighted_pick 함수를 정의합니다.
    def weighted_pick(weights):
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime       # 샘플링 결과를 리턴받을 ret 변수에 첫번째 글자를 할당합니다.
    char = prime[-1]   # Char-RNN의 첫번쨰 인풋을 지정합니다.  
    for n in range(num_sampling):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]

        # RNN을 한스텝 실행하고 Softmax 행렬을 리턴으로 받습니다.
        feed_dict = {input_data: x, state_batch_size : 1, initial_state: state}
        [probs_result, state] = sess.run([probs, final_state], feed_dict=feed_dict)         

        # 불필요한 차원을 제거합니다.
        # probs_result : (1,65) -> p : (65)
        p = np.squeeze(probs_result)

        # 샘플링 타입에 따라 3가지 종류로 샘플링 합니다.
        # sampling_type : 0 -> 다음 글자를 예측할때 항상 argmax를 사용
        # sampling_type : 1(defualt) -> 다음 글자를 예측할때 항상 random sampling을 사용
        # sampling_type : 2 -> 다음 글자를 예측할때 이전 글자가 ' '(공백)이면 random sampling, 그렇지 않을 경우 argmax를 사용
        if sampling_type == 0:
            sample = np.argmax(p)
        elif sampling_type == 2:
            if char == ' ':
                sample = weighted_pick(p)
            else:
                sample = np.argmax(p)
        else:
            sample = weighted_pick(p)

        pred = chars[sample]
        ret += pred     # 샘플링 결과에 현재 스텝에서 예측한 글자를 추가합니다. (예를들어 pred=L일 경우, ret = HEL -> HELL)
        char = pred     # 예측한 글자를 다음 RNN의 인풋으로 사용합니다.

    print("샘플링 결과:")
    print(ret)